In [2]:
import numpy as np
from math import e
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
import cvxopt as cvo
from cvxopt import matrix
import sklearn
from sklearn.utils import shuffle
from sklearn import svm

In [18]:
class CrossVal():
    
    def partitionData(self,splits=10):
        idxs = list(shuffle(self.train).index)
        result = np.array_split(idxs, 10) 
        return result
    
        
    def EvalForIdx(self,C,valIdxs):
        Xval = self.Xtrain.loc[valIdxs]
        Xtrain = self.Xtrain.loc[~self.Xtrain.index.isin(valIdxs)]
        yval = self.ytrain.loc[valIdxs]
        ytrain = self.ytrain.loc[~self.ytrain.index.isin(valIdxs)]
        
        
        my_svm = svm.SVC(C = C, kernel = 'poly',degree = 2, coef0 = 1.0, gamma = 1.0)
        my_svm.fit(Xtrain,ytrain)
        Eval = self.E(my_svm, Xval,yval)
        return Eval
    
    def runEval(self,partitions,C,splits=10):
        Evals = [] 
        for x in range(10):
            valIdxs = list(partitions[x])
            Eval = self.EvalForIdx(C,valIdxs)
            Evals.append(Eval)
        Eval = np.average(Evals)
        return Eval
            
    def crossVal(self):
        partitions = self.partitionData(splits=10)
        smallestEval = 1
        Evals = []
        for C in [0.0001, 0.001, 0.01, 0.1, 1]:
            Eval = self.runEval(partitions,C)
            #print("10 Fold Eval %s ; C is %s" % (Eval, C))
            Evals.append(Eval)
            if smallestEval > Eval:
                smallestEval = Eval
                self.finalC = C
        #print("Best C is %s with Eval of %s" % (self.finalC,smallestEval))
        self.avgEval = np.average(Evals)
        
    def E(self,my_svm,X,y):
        return 1-my_svm.score(X, y)   
            
    
class create_test(CrossVal):
    def __init__(self,val1,val2=None):
        self.val1 = val1
        self.val2 = val2
        
        val2 = "all" if self.val2==None else self.val2
        #print("%s vs. %s" % (self.val1, val2))
        
        self.train = pd.read_csv("../data/features.train.csv",index_col=0)
        self.train.columns = ["digit","x1","x2"]
        self.test = pd.read_csv("../data/features.test.csv",index_col=0)
        self.test.columns = ["digit","x1","x2"]
        
        self.initDataset()
        self.N = len(self.Xtrain)
        
        self.crossVal()
        
        
        
    def initDataset(self):
        if self.val2 != None:
            self.train = self.train[(self.train["digit"]==self.val1) | (self.train["digit"]==self.val2)]
            self.train.loc[self.train["digit"]==self.val1,"y"]=1
            self.train.loc[self.train["digit"]==self.val2,"y"]=-1
            
            self.test = self.test[(self.test["digit"]==self.val1) | (self.test["digit"]==self.val2)]
            self.test.loc[self.test["digit"]==self.val1,"y"]=1
            self.test.loc[self.test["digit"]==self.val2,"y"]=-1
        else:
            self.train["y"] = -1
            self.train.loc[self.train["digit"]==self.val1,"y"]=1
            
            self.test["y"] = -1
            self.test.loc[self.test["digit"]==self.val1,"y"]=1
        
        
        self.Xtrain = self.train[["x1","x2"]]
        self.ytrain = self.train.y
        self.Xtest = self.test[["x1","x2"]]
        self.ytest = self.test.y 
            
    

In [19]:
create_test(1,5)

1 vs. 5
10 Fold Eval 0.009615384615384592 ; C is 0.0001
10 Fold Eval 0.00512820512820511 ; C is 0.001
10 Fold Eval 0.005124122162338707 ; C is 0.01
10 Fold Eval 0.005124122162338707 ; C is 0.1
10 Fold Eval 0.004483096521313068 ; C is 1
Best C is 1 with Eval of 0.004483096521313068


In [7]:

finalCs = [create_test(1,5).finalC for x in range(10)]


In [13]:
unique, counts = np.unique(finalCs, return_counts=True)

list(np.asarray((unique, counts)).T)

[array([1.e-03, 6.e+00]),
 array([0.01, 2.  ]),
 array([0.1, 1. ]),
 array([1., 1.])]

In [16]:
avgEvals = []
for x in range(10):
    test = create_test(1,5)
    if test.finalC == .001:
        avgEval = test.avgEval
        avgEvals.append(avgEval)
np.average(avgEvals)

0.005680766508791971